Initial setup and first topic model run

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import wosis
import pandas as pd
import metaknowledge as mk

C:\Users\douglasd\AppData\Local\Continuum\miniconda3\envs\biblio\lib\site-packages\fuzzywuzzy-0.17.0-py3.6.egg\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Helper functions for plotting
import wosis.analysis.plotting as wos_plot

# Analysis and search functions
from wosis.analysis import (preview_matches, search_records, keyword_matches, 
                            find_pubs_by_authors, preview_matches_by_keyword, get_unique_kw_titles,
                            remove_by_journals, remove_by_title)

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# This just helps make the plots look nicer
sns.set_style('darkgrid')
sns.set_context('paper', font_scale=2.0)

In [4]:
# Read in data from temporary cache file
query_id = "756d39801152fe5f5f4ad3a3df9b6a30"
RC = mk.RecordCollection("tmp/{}.txt".format(query_id))

expected_num_records = 15754
num_records = len(RC)

assert num_records == expected_num_records, \
    "Mismatch in number of records - expected {}, got {}".format(expected_num_records, num_records)

print("Corpora consists of", num_records, "Publications")

Corpora consists of 15754 Publications


In [5]:
corpora_df = wosis.rc_to_df(RC)

corpora_df, removed_pubs = wosis.constrain.remove_empty_DOIs(corpora_df)
#maybe we don't want to do this for this analysis? older papers less likely to have dois?

Removing 1799 records with no DOIs


In [6]:
# removed_pubs

In [7]:
# Hiding keywords and abstract for tutorial (WoS-owned IP)
hide_columns = ['DE', 'abstract', 'keywords', 'DOI', 'id', 'kws']
corpora_df.loc[:, ~corpora_df.columns.isin(hide_columns)].head()  # display first 5

,year,title,copyright,AU,SO
0,2008,A simple algorithm for yield estimates: Evalua...,Elsevier Ltd. All rights reserved.,"Duchemin, Benoit|Maisongrande, Philippe|Boulet...",ENVIRONMENTAL MODELLING & SOFTWARE
1,2007,Estimating uncertainty in terrestrial critical...,(c) 2007 Elsevier B.V. All rights reserved.,"Skeffington, R. A.|Whitehead, P. G.|Heywood, E...",SCIENCE OF THE TOTAL ENVIRONMENT
3,2009,Regional analysis of the impacts of climate ch...,,"Bradley, Bethany A.",GLOBAL CHANGE BIOLOGY
4,2012,An integrated simulation-monitoring framework ...,(C) 2011 Published by Elsevier B. V. Selectio...,"Chen, L.|Liu, R. M.|Huang, Q.|Chen, Y. X.|Gao,...",18TH BIENNIAL ISEM CONFERENCE ON ECOLOGICAL MO...
5,2008,A polynomial dimensional decomposition for sto...,"Copyright (c) 2008 John Wiley & Sons, Ltd.","Rahman, Sharif",INTERNATIONAL JOURNAL FOR NUMERICAL METHODS IN...


In [8]:
# Create a new metaknowledge collection
filtered_corpora = wosis.extract_recs(corpora_df.loc[:, 'id'], RC, name='Filtered Corpora')

In [9]:
from wosis.analysis import find_topics

filtered_corpora_df = wosis.rec_to_df(filtered_corpora)
topic_results = find_topics(filtered_corpora_df, num_topics=5)

Topic 1: water model soil flow river parameters sensitivity groundwater runoff data
Topic 2: optimization design method shape topology sensitivity problem element finite structural
Topic 3: energy life cycle environmental emissions production waste assessment lca impact
Topic 4: uncertainty model models stochastic data analysis parameter parameters method bayesian
Topic 5: risk exposure assessment health cancer risks human flood probabilistic dose


In [10]:
topic_4_set = topic_results.get_topic_by_id(4)

In [11]:
topic_4_recs = wosis.extract_recs(topic_4_set.loc[:, 'id'], RC, 'Topic of Interest')

In [12]:
#run topic model on topic 4
topic_4_subtopics = find_topics(topic_4_set, num_topics=5)

C:\Users\douglasd\AppData\Local\Continuum\miniconda3\envs\biblio\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Topic 1: uncertainty model uncertainties analysis water models quality assessment results modeling
Topic 2: stochastic random polynomial chaos method quantification equations expansion collocation solution
Topic 3: sensitivity analysis model parameters global models based input parameter output
Topic 4: bayesian data model posterior parameter prior markov inference models estimation
Topic 5: fuzzy network decision neural ann criteria networks data artificial making


<hr>

Running key-phrase extraction on subtopics.

In [13]:
# Need config file
wos_config = wosis.load_config('config.yml')

In [14]:
subtopic1 = topic_4_subtopics.get_topic_by_id(1)
subtopic3 = topic_4_subtopics.get_topic_by_id(3)

In [15]:
uncertainty_model_rc = wosis.extract_recs(subtopic1['id'], RC, name='uncertainty_model')

In [16]:
sensitivity_analysis_rc = wosis.extract_recs(subtopic3['id'], RC, name='sensitivity_analysis')

In [17]:
# If cache directory is not specified, it will not save results for later re-use
uncertainty_model_cites = wosis.get_num_citations(uncertainty_model_rc, wos_config, cache_dir='tmp')
sensitivity_analysis_cites = wosis.get_num_citations(sensitivity_analysis_rc, wos_config, cache_dir='tmp')

In [18]:
uncertainty_model_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Refsgaard, Jens Christian|van der Sluijs, Jero...",10.1016/j.envost.2007.02.004,Uncertainty in the environmental modelling pro...,509
1,"Asseng, S.|Ewert, F.|Rosenzweig, C.|Jones, J. ...",10.1038/NCLIMATE1916,Uncertainty in simulating wheat yields under c...,448
2,"Gupta, Hoshin V.|Wagener, Thorsten|Liu, Yuqiong",10.1002/hyp.6989,Reconciling theory with observations: elements...,269
3,"Muleta, MK|Nicklow, JW",10.1016/j.jhydrol.2004.09.005,Sensitivity and uncertainty analysis coupled w...,259
4,"Olofsson, Pontus|Foody, Giles M.|Stehman, Step...",10.1016/j.rse.2012.10.031,Making better use of accuracy data in land cha...,251
5,"Refsgaard, Jens Christian|van der Sluijs, Jero...",10.1016/j.advwatres.2005.11.013,A framework for dealing with uncertainty due t...,246
6,"Pappenberger, F.|Beven, K. J.",10.1029/2005WR004820,Ignorance is bliss: Or seven reasons not to us...,244
7,"Aronica, G|Bates, PD|Horritt, MS",10.1002/hyp.398,Assessing the uncertainty in distributed model...,215
8,"Beier, Paul|Majka, Daniel R.|Spencer, Wayne D.",10.1111/j.1523-1739.2008.00942.x,Forks in the road: Choices in procedures for d...,208
9,"Pappenberger, F|Beven, K|Horritt, M|Blazkova, S",10.1016/j.jhydrol.2004.06.036,Uncertainty in the calibration of effective ro...,199


In [19]:
res1 = wosis.analysis.constrain.find_phrases(uncertainty_model_cites[0:10])

In [20]:
#phrases1 = res1.all_phrases
res1.display_phrases()

Uncertainty in the environmental modelling process - A framework and guidance (2007) 
https://dx.doi.org/10.1016/j.envost.2007.02.004 

Instead uncertainty should be seen as a red thread throughout the modelling study starting from the very beginning, where the identification and characterisation of all uncertainty sources should be performed jointly by the modeller, the water manager and the stakeholders. 


Uncertainty in simulating wheat yields under climate change (2013) 
https://dx.doi.org/10.1038/NCLIMATE1916 

Projections of climate change impacts on crop yields are inherently uncertain(). 

Here we present the largest standardized model intercomparison for climate change impacts so far. 

These impact uncertainties can be reduced by improving temperature and CO relationships in models and better quantified through use of multi-model ensembles. 

We found that individual crop models are able to simulate measured wheat grain yields accurately under a range of environments, partic

In [21]:
sensitivity_analysis_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Sobol, IM",10.1016/S0378-4754(00)00270-6,Global sensitivity indices for nonlinear mathe...,1223
1,"Saltelli, Andrea|Annoni, Paola|Azzini, Ivano|C...",10.1016/j.cpc.2009.09.018,Variance based sensitivity analysis of model o...,645
2,"Saltelli, A",10.1016/S0010-4655(02)00280-1,Making best use of model evaluations to comput...,566
3,"Saltelli, Andrea|Annoni, Paola",10.1016/j.envsoft.2010.04.012,How to avoid a perfunctory sensitivity analysis,353
4,"Brun, R|Reichert, P|Kunsch, HR",10.1029/2000WR900350,Practical identifiability analysis of large en...,278
5,"Brandtberg, T|Warner, TA|Landenberger, RE|McGr...",10.1016/S0034-4257(03)00008-7,Detection and analysis of individual leaf-off ...,242
6,"Kayitakire, F|Hamel, C|Defourny, P",10.1016/j.rse.2006.02.022,Retrieving forest structure variables based on...,185
7,"Patidar, Vinod|Pareek, N. K.|Sud, K. K.",10.1016/j.cnsns.2008.11.005,A new substitution-diffusion based image ciphe...,158
8,"Alis, OF|Rabitz, H",10.1023/A:1010979129659,Efficient implementation of high dimensional m...,157
9,"Helton, JC|Davis, FJ",10.1111/0272-4332.00041,Illustration of sampling-based methods for unc...,152


In [22]:
res2 = wosis.analysis.constrain.find_phrases(sensitivity_analysis_cites[0:10])

In [23]:
#phrases2 = res2.all_phrases
res2.display_phrases()

Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates (2001) 
https://dx.doi.org/10.1016/S0378-4754(00)00270-6 


Variance based sensitivity analysis of model output. Design and estimator for the total sensitivity index (2010) 
https://dx.doi.org/10.1016/j.cpc.2009.09.018 

as f(X- , X-, ... , X-k) is often in the form of a numerical model which may take long processing time. 

Practitioners can in principle describe the sensitivity pattern of a model Y = f (X- , X-, ... , X-k) with k uncertain input factors via a full decomposition of the variance V of Y into terms depending on the factors and their interactions. 

More often practitioners are satisfied with computing just k first order effects and k total effects, the latter describing synthetically interactions among input factors. 

In sensitivity analysis a key concern is the computational cost of the analysis, defined in terms of number of evaluations of f(X- , X-, ... , X-k) needed to compl

Illustration of sampling-based methods for uncertainty and sensitivity analysis (2002) 
https://dx.doi.org/10.1111/0272-4332.00041 

Uncertainty results obtained with replicated random and Latin hypercube samples are compared, with the Latin hypercube samples tending to produce more stable results than the random samples. 

A sequence of linear, monotonic, and nonmonotonic test problems is used to illustrate sampling-based uncertainty and sensitivity analysis procedures. 

PCCs, SRCs) performing well on the linear test problems, () measures based on rank transforms (i.e.. RCCs, PRCCs, SRRCs) performing well on the monotonic test problems, and () measures predicated on searches for nonrandom patterns (i.e., CMNs, CLs, CMDs, SI) performing well on the nonmonotonic test problems. 




<hr>
Running topic model on subtopics1,3

In [24]:
subtopic1_topics = find_topics(subtopic1, num_topics=5)

C:\Users\douglasd\AppData\Local\Continuum\miniconda3\envs\biblio\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Topic 1: model analysis uncertainties models data parameters carlo method monte using
Topic 2: model parameter calibration glue hydrological swat rainfall likelihood watershed runoff
Topic 3: flood inundation damage rainfall forecasting mapping river frequency maps extreme
Topic 4: urban modelling quality drainage stormwater water glue integrated sewer likelihood
Topic 5: air emission quality emissions pollution pollutant dispersion pm concentrations source


In [25]:
subtopic3_topics = find_topics(subtopic3, num_topics=5)

C:\Users\douglasd\AppData\Local\Continuum\miniconda3\envs\biblio\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Topic 1: model sensitivity parameters models parameter uncertainty based data simulation method
Topic 2: indices variance sobol sensitivity inputs global input output based measures
Topic 3: failure reliability probability kriging carlo monte method proposed probabilistic mcs
Topic 4: semantic lsa latent knowledge similarity vector space representation learning user
Topic 5: sampling latin hypercube monte carlo sample random samples number methods


In [26]:
sub1subtopic1 = subtopic1_topics.get_topic_by_id(1)
sub1subtopic2 = subtopic1_topics.get_topic_by_id(2)

In [27]:
sub3subtopic1 = subtopic3_topics.get_topic_by_id(1)
sub3subtopic2 = subtopic3_topics.get_topic_by_id(2)
sub3subtopic5 = subtopic3_topics.get_topic_by_id(5)

<hr>
Running key-phrase extraction on the subtopics.

In [28]:
model_analysis_rc = wosis.extract_recs(sub1subtopic1['id'], RC, name='model_analysis')
model_param_rc = wosis.extract_recs(sub1subtopic2['id'], RC, name='model_param')

model_sens_rc = wosis.extract_recs(sub3subtopic1['id'], RC, name='model_sens')
indices_var_rc = wosis.extract_recs(sub3subtopic2['id'], RC, name='indices_var')
sampling_rc = wosis.extract_recs(sub3subtopic5['id'], RC, name='sampling')

In [29]:
# If cache directory is not specified, it will not save results for later re-use
model_analysis_cites = wosis.get_num_citations(model_analysis_rc, wos_config, cache_dir='tmp')
model_param_cites = wosis.get_num_citations(model_param_rc, wos_config, cache_dir='tmp')

model_sens_cites = wosis.get_num_citations(model_sens_rc, wos_config, cache_dir='tmp')
indices_var_cites = wosis.get_num_citations(indices_var_rc, wos_config, cache_dir='tmp')
sampling_cites = wosis.get_num_citations(sampling_rc, wos_config, cache_dir='tmp')

In [30]:
model_analysis_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Asseng, S.|Ewert, F.|Rosenzweig, C.|Jones, J. ...",10.1038/NCLIMATE1916,Uncertainty in simulating wheat yields under c...,448
1,"Gupta, Hoshin V.|Wagener, Thorsten|Liu, Yuqiong",10.1002/hyp.6989,Reconciling theory with observations: elements...,269
2,"Olofsson, Pontus|Foody, Giles M.|Stehman, Step...",10.1016/j.rse.2012.10.031,Making better use of accuracy data in land cha...,251
3,"Refsgaard, Jens Christian|van der Sluijs, Jero...",10.1016/j.advwatres.2005.11.013,A framework for dealing with uncertainty due t...,246
4,"Pappenberger, F.|Beven, K. J.",10.1029/2005WR004820,Ignorance is bliss: Or seven reasons not to us...,244
5,"Beier, Paul|Majka, Daniel R.|Spencer, Wayne D.",10.1111/j.1523-1739.2008.00942.x,Forks in the road: Choices in procedures for d...,208
6,"Bond, Alexander L.|Diamond, Antony W.",10.1890/09-2409.1,Recent Bayesian stable-isotope mixing models a...,194
7,"Oberkampf, William L.|Barone, Matthew F.",10.1016/j.jcp.2006.03.037,Measures of agreement between computation and ...,173
8,"Runge, Michael C.|Converse, Sarah J.|Lyons, Ja...",10.1016/j.biocon.2010.12.020,Which uncertainty? Using expert elicitation an...,170
9,"Roy, Christopher J.|Oberkampf, William L.",10.1016/j.cma.2011.03.016,"A comprehensive framework for verification, va...",167


In [31]:
res_model_analysis = wosis.analysis.constrain.find_phrases(model_analysis_cites[0:10])

In [32]:
#phrases1 = res1.all_phrases
res_model_analysis.display_phrases()

Uncertainty in simulating wheat yields under climate change (2013) 
https://dx.doi.org/10.1038/NCLIMATE1916 

Projections of climate change impacts on crop yields are inherently uncertain(). 

Here we present the largest standardized model intercomparison for climate change impacts so far. 

These impact uncertainties can be reduced by improving temperature and CO relationships in models and better quantified through use of multi-model ensembles. 

We found that individual crop models are able to simulate measured wheat grain yields accurately under a range of environments, particularly if the input information is sufficient. 


Reconciling theory with observations: elements of a diagnostic approach to model evaluation (2008) 
https://dx.doi.org/10.1002/hyp.6989 

and can be readily applied to the problem of process evaluation leading to improved predictions in ungaged basins. 

and steps are proposed towards the development of a robust and powerful 'Theory of Evaluation'. 

This paper

The framework is comprehensive in the sense that it treats both types of uncertainty (aleatory and epistemic), incorporates uncertainty due to the mathematical form of the model, and it provides a procedure for including estimates of numerical error in the predictive uncertainty. 

An overview of a comprehensive framework is given for estimating the predictive uncertainty of scientific computing applications. 

Finally, methods for conveying the total predictive uncertainty to decision makers are presented. 

Approaches for propagating both types of uncertainties through the model to the system response quantities of interest are briefly discussed. 




In [33]:
model_param_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Muleta, MK|Nicklow, JW",10.1016/j.jhydrol.2004.09.005,Sensitivity and uncertainty analysis coupled w...,259
1,"Stedinger, Jery R.|Vogel, Richard M.|Lee, Seun...",10.1029/2008WR006822,Appraisal of the generalized likelihood uncert...,137
2,"Vrugt, Jasper A.|Robinson, Bruce A.",10.1029/2005WR004838,Treatment of uncertainty using ensemble method...,128
3,"Gallagher, Mark|Doherty, John",10.1016/j.envsoft.2006.06.007,Parameter estimation and uncertainty analysis ...,118
4,"Zhang, Xuesong|Srinivasan, Raghavan|Bosch, David",10.1016/j.jhydrol.2009.06.023,Calibration and uncertainty analysis of the SW...,112
5,"Blasone, Roberta-Serena|Madsen, Henrik|Rosbjer...",10.1016/j.jhydrol.2007.12.026,Uncertainty assessment of integrated distribut...,108
6,"Hong, Yang|Hsu, Kuo-lin|Moradkhani, Hamid|Soro...",10.1029/2005WR004398,Uncertainty quantification of satellite precip...,99
7,"Strauch, Michael|Bernhofer, Christian|Koide, S...",10.1016/j.jhydrol.2011.11.014,Using precipitation data ensemble for uncertai...,81
8,"Kavetski, D|Kuczera, G|Franks, SW",10.1016/j.jhydrol.2005.07.012,Calibration of conceptual hydrological models ...,78
9,"Shen, Zhenyao|Hong, Qian|Yu, Hong",10.1016/j.scitotenv.2008.06.009,Parameter uncertainty analysis of the non-poin...,75


In [34]:
res_model_param = wosis.analysis.constrain.find_phrases(model_param_cites[0:10])

In [35]:
#phrases2 = res2.all_phrases
res_model_param.display_phrases()

Sensitivity and uncertainty analysis coupled with automatic calibration for a distributed watershed model (2005) 
https://dx.doi.org/10.1016/j.jhydrol.2004.09.005 

To demonstrate their effectiveness, the parameter identification, parameter estimation, model verification, and uncertainty analysis techniques are applied to a watershed located in southern Illinois. 

The global parameter sensitivity analysis is conducted using a stepwise regression analysis on rank-transformed input-output data pairs. 

The Generalized Likelihood Uncertainty Estimation methodology is subsequently implemented to investigate uncertainty of model estimates, accounting for errors due to model structure, input data and model parameters. 


Appraisal of the generalized likelihood uncertainty estimation (GLUE) method (2008) 
https://dx.doi.org/10.1029/2008WR006822 

Beven and others have suggested that the choice of the likelihood measure used in a GLUE computation is subjective and may be selected to reflect t

The uncertainty of runoff was smaller compared to that of sediment, organic N, nitrate, and TP, and the source of uncertainty was mainly affected by parameters associated with runoff. 

In this study, three methods were adopted to do the parameter uncertainty analysis with the Soil and Water Assessment Tool (SWAT). 

Based on the results of parameter sensitivity analysis by the Morris screening method, the ten parameters that most affect runoff, sediment, organic N, nitrate, and total phosphorous (TP) were chosen for further uncertainty analysis. 

MC results indicated that neglecting the parameter uncertainty of the model would underestimate the non-point source pollution load, and that the relationship between model input and output was non-linear. 




In [36]:
model_sens_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Saltelli, Andrea|Annoni, Paola",10.1016/j.envsoft.2010.04.012,How to avoid a perfunctory sensitivity analysis,353
1,"Brun, R|Reichert, P|Kunsch, HR",10.1029/2000WR900350,Practical identifiability analysis of large en...,278
2,"Brandtberg, T|Warner, TA|Landenberger, RE|McGr...",10.1016/S0034-4257(03)00008-7,Detection and analysis of individual leaf-off ...,242
3,"Kayitakire, F|Hamel, C|Defourny, P",10.1016/j.rse.2006.02.022,Retrieving forest structure variables based on...,185
4,"Patidar, Vinod|Pareek, N. K.|Sud, K. K.",10.1016/j.cnsns.2008.11.005,A new substitution-diffusion based image ciphe...,158
5,"Pianosi, Francesca|Beven, Keith|Freer, Jim|Hal...",10.1016/j.envsoft.2016.02.008,Sensitivity analysis of environmental models: ...,131
6,"Filatova, Tatiana|Verburg, Peter H.|Parker, Da...",10.1016/j.envsoft.2013.03.017,Spatial agent-based models for socio-ecologica...,127
7,"Crosetto, M|Tarantola, S|Saltelli, A",10.1016/S0167-8809(00)00169-9,Sensitivity and uncertainty analysis in spatia...,122
8,"Hall, JW|Tarantola, S|Bates, PD|Horritt, MS",10.1061/(ASCE)0733-9429(2005)131:2(117),Distributed sensitivity analysis of flood inun...,120
9,"Yang, Jing",10.1016/j.envsoft.2010.10.007,Convergence and uncertainty analyses in Monte-...,120


In [37]:
res_model_sens = wosis.analysis.constrain.find_phrases(model_sens_cites[0:10])

In [38]:
#phrases2 = res2.all_phrases
res_model_sens.display_phrases()

How to avoid a perfunctory sensitivity analysis (2010) 
https://dx.doi.org/10.1016/j.envsoft.2010.04.012 

Mathematical modelers from different disciplines and regulatory agencies worldwide agree on the importance of a careful sensitivity analysis (SA) of model-based inference. 

The most popular SA practice seen in the literature is that of 'one-factor-at-a-time' (OAT). 

The present paper introduces a novel geometric proof of the inefficiency of OAT, with the purpose of providing the modeling community with a convincing and possibly definitive argument against OAT. 

This consists of analyzing the effect of varying one model input factor at a time while keeping all other fixed. 


Practical identifiability analysis of large environmental simulation models (2001) 
https://dx.doi.org/10.1029/2000WR900350 

It is shown how these measures provide identifiability diagnosis for parameter subsets, how they are able to guide the selection of identifiable parameter subsets for parameter estim

In [39]:
indices_var_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Sobol, IM",10.1016/S0378-4754(00)00270-6,Global sensitivity indices for nonlinear mathe...,1223
1,"Saltelli, Andrea|Annoni, Paola|Azzini, Ivano|C...",10.1016/j.cpc.2009.09.018,Variance based sensitivity analysis of model o...,645
2,"Saltelli, A",10.1016/S0010-4655(02)00280-1,Making best use of model evaluations to comput...,566
3,"Alis, OF|Rabitz, H",10.1023/A:1010979129659,Efficient implementation of high dimensional m...,157
4,"Borgonovo, Emanuele",10.1111/j.1539-6924.2006.00806.x,Measuring uncertainty importance: Investigatio...,139
5,"Ziehn, T.|Tomlin, A. S.",10.1016/j.envsoft.2008.12.002,GUI-HDMR - A software tool for global sensitiv...,120
6,"Kucherenko, S.|Tarantola, S.|Annoni, P.",10.1016/j.cpc.2011.12.020,Estimation of global sensitivity indices for m...,102
7,"Tate, Eric",10.1007/s11069-012-0152-2,Social vulnerability indices: a comparative as...,95
8,"Wainwright, Haruko M.|Finsterle, Stefan|Jung, ...",10.1016/j.cageo.2013.06.006,Making sense of global sensitivity analyses,71
9,"Borgonovo, Emanuele|Castaings, William|Taranto...",10.1111/j.1539-6924.2010.01519.x,Moment Independent Importance Measures: New Re...,63


In [40]:
res_indices_var = wosis.analysis.constrain.find_phrases(indices_var_cites[0:10])

In [41]:
#phrases2 = res2.all_phrases
res_indices_var.display_phrases()

Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates (2001) 
https://dx.doi.org/10.1016/S0378-4754(00)00270-6 


Variance based sensitivity analysis of model output. Design and estimator for the total sensitivity index (2010) 
https://dx.doi.org/10.1016/j.cpc.2009.09.018 

as f(X- , X-, ... , X-k) is often in the form of a numerical model which may take long processing time. 

Practitioners can in principle describe the sensitivity pattern of a model Y = f (X- , X-, ... , X-k) with k uncertain input factors via a full decomposition of the variance V of Y into terms depending on the factors and their interactions. 

More often practitioners are satisfied with computing just k first order effects and k total effects, the latter describing synthetically interactions among input factors. 

In sensitivity analysis a key concern is the computational cost of the analysis, defined in terms of number of evaluations of f(X- , X-, ... , X-k) needed to compl

In [42]:
sampling_cites[0:10].loc[:, ["AU", "DOI", 'title', 'citations']]

,AU,DOI,title,citations
0,"Helton, JC|Davis, FJ",10.1111/0272-4332.00041,Illustration of sampling-based methods for unc...,152
1,"Ratto, Marco|Pagano, Andrea|Young, Peter",10.1016/j.cpc.2007.07.011,State dependent parameter metamodelling and se...,134
2,"Hupet, F|Vanclooster, M",10.1016/S0022-1694(00)00413-3,Effect of the sampling frequency of meteorolog...,78
3,"Gan, Yanjun|Duan, Qingyun|Gong, Wei|Tong, Char...",10.1016/j.envsoft.2013.09.031,A comprehensive evaluation of various sensitiv...,72
4,"Manache, Gemma|Melching, Charles S.",10.1016/j.envsoft.2007.08.001,Identification of reliable regression- and cor...,71
5,"Mathis, U|Ristimaki, J|Mohr, M|Keskinen, J|Ntz...",10.1080/027868290891497,Sampling conditions for the measurement of nuc...,64
6,"Isukapalli, SS|Roy, A|Georgopoulos, PG",10.1111/0272-4332.205054,Efficient sensitivity/uncertainty analysis usi...,50
7,"Castaings, W.|Borgonovo, E.|Morris, M. D.|Tara...",10.1016/j.envsoft.2012.04.017,Sampling strategies in density-based sensitivi...,39
8,"Ruano, M. V.|Ribes, J.|Seco, A.|Ferrer, J.",10.1016/j.envsoft.2012.03.008,An improved sampling strategy based on traject...,37
9,"Lu, ZM|Zhang, DX",10.1016/S0309-1708(03)001060-4,On importance sampling Monte Carlo approach to...,28


In [43]:
res_sampling = wosis.analysis.constrain.find_phrases(sampling_cites[0:10])

In [44]:
#phrases2 = res2.all_phrases
res_sampling.display_phrases()

Illustration of sampling-based methods for uncertainty and sensitivity analysis (2002) 
https://dx.doi.org/10.1111/0272-4332.00041 

     Uncertainty results obtained with replicated random and Latin hypercube samples are compared, with the Latin hypercube samples tending to produce more stable results than the random samples. 

     A sequence of linear, monotonic, and nonmonotonic test problems is used to illustrate sampling-based uncertainty and sensitivity analysis procedures. 

     PCCs, SRCs) performing well on the linear test problems, () measures based on rank transforms (i.e.. RCCs, PRCCs, SRRCs) performing well on the monotonic test problems, and () measures predicated on searches for nonrandom patterns (i.e., CMNs, CLs, CMDs, SI) performing well on the nonmonotonic test problems. 


State dependent parameter metamodelling and sensitivity analysis (2007) 
https://dx.doi.org/10.1016/j.cpc.2007.07.011 

     The method is conceptually simple and very efficient, leading to a si